In [56]:
%display latex

In [57]:
M = Manifold(2, 'M',r'\mathcal{M}', start_index=1)

Our manifold is going to be the sphere with a cut from the north pole to the south pole and we can cover this with a single chart.

In [58]:
X.<x,y> = M.chart()

In [59]:
print(M)

2-dimensional differentiable manifold M


Create a connection with as yet no connection co-efficients.

In [60]:
nab = M.affine_connection('nabla', latex_name=r'\nabla')


In [61]:
ch_basis = M.automorphism_field()

In [62]:
ch_basis[1,1], ch_basis[2,2] = 1, 1/sin(x)

In [63]:
e = M.default_frame().new_frame(ch_basis, 'e')

In [64]:
nab.add_coef(e)


3-indices components w.r.t. Vector frame (M, (e_1,e_2))

In [65]:
print(nab.connection_form(1,1,e)[:])

[0, 0]


In [66]:
print(nab.connection_form(1,2,e)[:])

[0, 0]


In [67]:
print(nab.connection_form(2,1,e)[:])

[0, 0]


In [68]:
print(nab.connection_form(2,2,e)[:])

[0, 0]


In [69]:
eX = X.frame()

In [70]:
print(nab.connection_form(1,1,eX)[:])

[0, 0]


In [71]:
print(nab.connection_form(1,2,eX)[:])

[0, 0]


In [72]:
print(nab.connection_form(2,1,eX)[:])

[0, 0]


In [73]:
print(nab.connection_form(2,2,eX)[:])

[cos(x)/sin(x), 0]
